In [1]:
import pandas as pd
import numpy as np
import json

## Loading "abm" and "abf"

In [2]:
abm = pd.read_csv("../Data/account_based_measures.csv")
abm.head()

,user_name,Twitter Account Duration(till 26/1/2020),Avg. Tweet Rate(per Month),Avg. Retweet Rate(per Month),Avg. Reply Rate(per Month),Total Normal Tweets,Total Retweets,Total Replies,Total Quotes,Avg. Fav Count(per Month),Avg. Retweeted Count(per Month),Avg. Quote Count(per Month)
0,001Danish,328285108.0,4.300,7.745,2.435,215,364,56,188,225.533,37526.150,4.947
1,0thFromOurStar,55209782.0,9.846,3.308,21.529,128,43,366,37,3501.391,1295.696,3.364
2,0x0000EBFE,223236506.0,2.087,5.988,1.719,48,503,55,4,5.386,1159.159,1.000
3,0xbharath,289685006.0,5.493,5.349,2.974,412,337,116,25,38.348,1252.663,1.667
4,0xdekster,181467531.0,7.057,9.000,14.688,247,243,470,71,119.452,11512.976,3.227


In [3]:
abf = pd.read_csv("../Data/account_based_features.csv")
abf.head()

,User ID,User Name,Followers,Following,Follower/Following Ratio,Total Status,Retweet Count,Total Favourites
0,2615733620,1000DegreesX,148,136,1.088,504,44,109
1,437763875,100Security,949,2856,0.332,549,2,372
2,269513390,121giving,3205,3286,0.975,617,78,95
3,3005258262,138_das,69,42,1.643,618,292,296
4,2160172507,13kamilkhan,3518,172,20.453,704,176,2738


## Loading friend list

In [4]:
with open('../Data/friends_library.json', 'r') as openfile:
    # Reading from json file
    json_dict = json.loads(openfile.read())

## Selecting user: "13kamilkhan"

In [17]:
username = "13kamilkhan"

# SIR

In [37]:
ndays = 30
dt = 0.0417 #time step in days
npts = int(ndays/dt)

S = np.zeros(npts) #susceptible
I = np.zeros(npts) #infective
# R = np.zeros(npts) # removed
t = np.arange(npts)*dt

#beta : infection rate
beta = abm[abm['user_name'] == username]['Avg. Tweet Rate(per Month)'].values[0]/30
gamma = (1.0/21.0) #recovery rate
N = abf[abf['User Name'] == username]['Followers'].values[0]

I[0] = 2 #initial infective proportion
S[0] = N - I[0] #initial susceptible
# R[0] = 0

score = dt
for i in range(npts-1):
    ds = int(S[i] - beta*(S[i]*I[i])*dt)
    di = int(I[i] + (beta*S[i]*I[i]-gamma*I[i])*dt)
    
    if di >= N:
        S[i+1] = 0
        I[i+1] = N
        print("S: ", S[i])
        print("I: ", I[i])
        break
    else:
        S[i+1] = ds
        I[i+1] = di
        
    print("S: ", S[i])
    print("I: ", I[i])
#         print("R: ", R[i])

S:  3516.0
I:  2.0
S:  3453.0
I:  64.0
S:  1483.0
I:  2032.0


In [52]:
I[1]

64.0

# Calculating score

In [45]:
score = 0
for i in I:
    score += dt
    if i == N:
        break
score

0.1668

# created the funtion to get sir score

In [9]:
usernames = abf['User Name'].tolist()
print(len(usernames))
for i in usernames:
    if i not in abm['user_name'].tolist():
        usernames.remove(i)
print(len(usernames))

8296
8296


In [10]:
ndays = 30
dt = 0.0417 #time step in days
npts = int(ndays/dt)


def calculate_score(username):
    S = np.zeros(npts) #susceptible
    I = np.zeros(npts) #infective
#     R = np.zeros(npts) # removed
#     t = np.arange(npts)*dt

    #beta : infection rate
    beta = abm[abm['user_name'] == username]['Avg. Tweet Rate(per Month)'].values[0]/30
    gamma = (1.0/21.0) #recovery rate
    N = abf[abf['User Name'] == username]['Followers'].values[0] # initial population

    I[0] = 2 #initial infective proportion
    S[0] = N - I[0] #initial susceptible
    # R[0] = 0

    for i in range(npts-1):        
        S[i+1] = int(S[i] - beta*(S[i]*I[i])*dt)
        I[i+1] = int(I[i] + (beta*S[i]*I[i]-gamma*I[i])*dt)
    #     R[i+1] = int(R[i] + (gamma*I[i])*dt)
        if I[i] >= N:
            I[i+1] = N
            break
#         print("S: ", S[i])
#         print("I: ", I[i])
#         print("R: ", R[i])
    score = 0
    for i in I:
        score += dt
        if i == N:
            break
    return score

In [11]:
import csv

with open('./sir.csv', 'w', newline='') as csvfile:
    fieldnames = ['username', 'score']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for i in usernames:
        try:
            score = calculate_score(i)
            print(i, "-->" , score)
            writer.writerow({'username': i, 'score': score})
        except Exception as e:
            print(e)   

1000DegreesX --> 0.25020000000000003
100Security --> 0.20850000000000002
121giving --> 0.1668
138_das --> 29.982299999999643
13kamilkhan --> 0.20850000000000002
13pt --> 0.20850000000000002
14Swathi --> 0.20850000000000002
18Bodaji --> 0.29190000000000005
19thCenturyShop --> 0.20850000000000002
19thmilecapsol --> 0.20850000000000002
207teacherguy --> 0.20850000000000002
23septemberrrr --> 0.20850000000000002
24namratha --> 0.20850000000000002
321Foundation --> 0.20850000000000002
40MILLIONBUCKS --> 0.1668
4WeddingsOnHulu --> 0.1668
4WorkerRights --> 0.20850000000000002
501NotOut --> 0.20850000000000002
5aelo --> 0.20850000000000002
5h1vang --> 0.29190000000000005
5onyaHorton --> 0.29190000000000005
5SmL9e5kpyuincH --> 0.20850000000000002
609402 --> 0.1668
60_decibels --> 0.1668
62f60ca138454a3 --> 29.982299999999643
63Jbk14zFFjBg4O --> 0.25020000000000003
6thFromOurStar --> 0.1668
777kabra --> 29.982299999999643
90sWWE --> 0.1668
9Harsh9_Shah --> 0.20850000000000002
9KDesigns --> 0.166

agnextworld --> 0.20850000000000002
AGoldmund --> 0.1668
agrawalmanindra --> 0.1668
agrawalsmart7 --> 0.20850000000000002
AgrawalVerma --> 0.29190000000000005
agrawal_aga --> 29.982299999999643
Agrawal_I --> 29.982299999999643
agrwlANUPRIYA --> 29.982299999999643
agulati115 --> 0.1668
Agunk_Apology --> 0.1668
Agusujung --> 0.3753000000000001
agzmn --> 0.20850000000000002
AG_Padmanaban --> 0.20850000000000002
AHASouthAsia --> 0.29190000000000005
aheineike --> 0.20850000000000002
AhmadouLY1 --> 29.982299999999643
ahmedraunaq --> 0.25020000000000003
ahmillar9 --> 0.25020000000000003
ahnis --> 0.20850000000000002
AhujaHemanya --> 0.29190000000000005
AICongressVegas --> 0.1668
aic_banasthali --> 0.20850000000000002
AidaE --> 0.25020000000000003
aIDEA_NAARM --> 0.20850000000000002
aiims_rpr --> 0.1668
aijazn --> 0.29190000000000005
AILA_Community --> 0.1668
aileenvlweb --> 0.20850000000000002
aimcavinay --> 0.20850000000000002
ainuindia --> 0.20850000000000002
AirAkshar --> 0.291900000000000

AndrewHardy6 --> 0.3753000000000001
andrewlissimore --> 0.25020000000000003
AndrewPhamzer --> 0.25020000000000003
AndrewTWriter --> 0.1668
AndrewZeitlin --> 0.29190000000000005
Andrew_S_Dykes --> 0.1668
andriheidar --> 0.20850000000000002
andrinhopereira --> 0.1668
AndSlavery --> 0.20850000000000002
andy23tran --> 0.20850000000000002
AndyGWood1 --> 0.1668
andykarmakar --> 29.982299999999643
andylassner --> 0.1668
AndySlatz --> 0.20850000000000002
AndyStoneA --> 0.25020000000000003
AndyTCarroll --> 0.1668
Andy_Plays_ --> 0.1668
aneeshabaig --> 0.25020000000000003
angad_virk --> 29.982299999999643
AngelaStent --> 0.20850000000000002
angelinaksun --> 29.982299999999643
AngeloCassisa --> 0.25020000000000003
ANGELZOFFICIAL --> 0.1668
AngilaLumumba --> 29.982299999999643
angiperera --> 0.1668
angira_biswas --> 0.20850000000000002
angira_dhar --> 0.1668
AngryChefASMR --> 0.1668
anichakladar --> 0.20850000000000002
anideshpandelab --> 0.29190000000000005
aniketdivekar --> 0.20850000000000002
a

AseemDokania --> 0.20850000000000002
ASethii --> 0.25020000000000003
asfaCW --> 0.29190000000000005
Asfak44511862 --> 29.982299999999643
ash647 --> 0.29190000000000005
AshaKanwarCOL --> 0.20850000000000002
ashankardass --> 0.20850000000000002
AshaPoulose --> 29.982299999999643
asher_ghertner --> 0.20850000000000002
asheth99 --> 0.25020000000000003
AshGowariker --> 0.1668
AshGupta76 --> 0.20850000000000002
ASHISH225 --> 0.20850000000000002
ashish22march --> 0.1668
AshishFafadia --> 0.25020000000000003
ashishjaiswaled --> 0.20850000000000002
ashishkpatra --> 0.20850000000000002
ashishrmohan --> 0.20850000000000002
ashishthacker --> 0.29190000000000005
ashish_vt --> 0.25020000000000003
AshmitPatel15 --> 29.982299999999643
AshokGalla_ --> 0.20850000000000002
ashokgchandran --> 0.25020000000000003
AshokKartham --> 0.25020000000000003
AshokKmrPandit --> 29.982299999999643
AshokMaruthi --> 0.20850000000000002
AshokMe56222876 --> 0.1668
ashokpamidi --> 0.33360000000000006
AshokPati25 --> 0.208

benlock95 --> 0.1668
benlpiper --> 0.25020000000000003
benmonreal --> 0.29190000000000005
BenoitKornmann --> 0.20850000000000002
BenRDawson --> 0.20850000000000002
BenSteventon2 --> 0.29190000000000005
benwynkoop1 --> 0.1668
beondeck --> 0.1668
berendennedy --> 29.982299999999643
BereniceFamili --> 0.25020000000000003
berkustun --> 0.25020000000000003
berlinf --> 0.25020000000000003
bernease --> 0.25020000000000003
bertcast --> 0.1668
Best_SEO_India --> 0.25020000000000003
betamarketer --> 0.25020000000000003
bethardCX --> 0.25020000000000003
BethFordLOL --> 0.1668
BethLewWilliams --> 0.20850000000000002
bethmorgan22 --> 0.25020000000000003
BethWild7 --> 0.20850000000000002
better18 --> 0.1668
bettymonex --> 0.20850000000000002
bevyshopping --> 0.20850000000000002
BeynonMD --> 0.20850000000000002
be_stranger --> 0.20850000000000002
bfcobert --> 0.25020000000000003
BFreeFrom_ --> 0.1668
bgarlinghouse --> 0.1668
BGMKamarhati --> 0.20850000000000002
bguptabio --> 29.982299999999643
Bhadan

...

soniasingh_13 --> 0.29190000000000005
sonicsarena --> 0.1668
soni_sori --> 0.20850000000000002
SonjaBPeterson --> 0.29190000000000005
sonulamba4 --> 29.982299999999643
sonumaithil --> 0.20850000000000002
SonuSah05276984 --> 29.982299999999643
SonuShivdasani --> 0.1668
sonzalp --> 29.982299999999643
SooDew --> 0.20850000000000002
SoodVandna --> 0.33360000000000006
soorajbhalla --> 0.33360000000000006
sooraj_kamath --> 0.29190000000000005
sooriofficial --> 0.1668
soosairajmichal --> 0.20850000000000002
SopanDeb --> 0.1668
sophdevine77 --> 0.1668
SophieJusticeW --> 0.25020000000000003
SOPMedUSA --> 0.20850000000000002
SoprAboist --> 0.29190000000000005
SorayyaKhan --> 0.25020000000000003
sortizreina --> 0.33360000000000006
souadmassioff --> 0.1668
Soulfulsachin --> 0.1668
SoumyaMukerji_ --> 0.25020000000000003
SoumyaShetty25 --> 0.29190000000000005
soumyavajpayee --> 0.25020000000000003
Soundarya_J --> 0.20850000000000002
souravact --> 0.1668
Souravi8 --> 29.982299999999643
souravsarkar03

Sunriseclick --> 0.20850000000000002
suntroncorp --> 0.1668
sunummm --> 0.20850000000000002
Suparna_Aangan --> 0.33360000000000006
SuperTalkTweets --> 0.20850000000000002
super_tech007 --> 29.982299999999643
suppcoolgayatri --> 0.20850000000000002
surajdidwania --> 0.25020000000000003
surajeet_ghosh --> 0.33360000000000006
surajkmurthy --> 29.982299999999643
SureshGorakala --> 0.20850000000000002
surfsquire --> 0.29190000000000005
surgeonwhocodes --> 29.982299999999643
SurgicalTheater --> 0.20850000000000002
surinde97727889 --> 29.982299999999643
SURINDERsKOHLI --> 29.982299999999643
surishining --> 29.982299999999643
Surjeet23598645 --> 29.982299999999643
Surrender_hs --> 0.20850000000000002
Susan_09 --> 0.29190000000000005
sushantmadan --> 0.1668
sushantmarathe --> 0.20850000000000002
SushantTaranal --> 29.982299999999643
Susheel_karam --> 0.29190000000000005
SushiHack --> 0.20850000000000002
SushilBaghel81 --> 0.25020000000000003
sushilborde --> 29.982299999999643
sushilchopra149 --

theshiftseries --> 0.20850000000000002
theshrutster --> 0.25020000000000003
TheSillyPoint_ --> 0.20850000000000002
TheSilverLab --> 0.25020000000000003
thesoul_traders --> 0.20850000000000002
TheStudyCastle --> 0.20850000000000002
thesulk --> 0.1668
TheTanayJain --> 0.3753000000000001
TheTannedDon --> 0.29190000000000005
theterenceyung --> 0.20850000000000002
ThethBiharan --> 0.25020000000000003
thetiehub --> 0.29190000000000005
TheTrickRoom --> 0.25020000000000003
thetrippytoes --> 0.20850000000000002
thevenon --> 0.20850000000000002
TheVivaanShah --> 0.1668
TheZoomStudios --> 0.1668
the_aliciasouza --> 0.20850000000000002
the_ankitas --> 0.25020000000000003
the_epicdad --> 0.1668
the_spearman --> 0.1668
thiagosantoss40 --> 0.1668
thibaultmathieu --> 0.20850000000000002
thierryzoller --> 0.1668
thinkingmike --> 0.25020000000000003
thiri66 --> 0.1668
ThirstKart --> 0.1668
ThisBeKenny --> 0.20850000000000002
ThisisBobcat --> 0.1668
thisisideadrop --> 0.20850000000000002
thisisitdel --> 

VidhuAggarwal --> 0.29190000000000005
vidursawhney --> 0.1668
vidyaiyengarblr --> 0.29190000000000005
viewsofvineet --> 0.29190000000000005
VigneshJayanth1 --> 0.25020000000000003
vihangsarnaik --> 0.20850000000000002
vijayan04 --> 0.20850000000000002
vijaychandru --> 0.20850000000000002
VijaygiriBava --> 0.20850000000000002
vijaypalande11 --> 0.20850000000000002
VijaySethuOffl --> 0.1668
VijaySinghGolf --> 0.1668
Vijay_Amritraj --> 0.1668
vijay_dutt --> 29.982299999999643
vijay_mkn --> 0.33360000000000006
vijay_r_kumar --> 0.20850000000000002
VijooKrishnan --> 0.1668
vikasbajaj --> 0.1668
Vikasguptabjp --> 0.20850000000000002
vikasjournalist --> 0.20850000000000002
vikasnitr --> 29.982299999999643
Vikasroy3273213 --> 29.982299999999643
Vikas_M_Sharma --> 29.982299999999643
vikchaudhary --> 0.25020000000000003
VIKME --> 0.3753000000000001
vikram0285 --> 0.20850000000000002
vikramsahney --> 0.1668
VikramVinod12 --> 0.29190000000000005
vikram_hockey --> 29.982299999999643
vikrplkh --> 29

xgEGsL0b9FV2TgT --> 0.20850000000000002
Xin426 --> 0.1668
xkk1995 --> 29.982299999999643
xodnd2GJ --> 0.1668
XpengMotors --> 0.1668
XSSExploits --> 0.20850000000000002
xtrememelychill --> 0.29190000000000005
xwang_lk --> 0.20850000000000002
yaariyan_lgbtq --> 0.20850000000000002
YadavRajeevSin1 --> 29.982299999999643
yadav_cheshta --> 0.20850000000000002
yadthore --> 29.982299999999643
YagyaSachdeva --> 0.20850000000000002
YaleNeuro --> 0.20850000000000002
YamahaFascinoIN --> 0.1668
yamalavyas --> 0.20850000000000002
YamaVelasco --> 0.25020000000000003
yama_shigechan --> 0.1668
yaminichan --> 0.20850000000000002
Yankee6Wickets --> 0.1668
yashadeshgupta --> 0.1668
YashrajKshetry --> 29.982299999999643
YashwantSinha --> 0.1668
yashwant_nahar1 --> 29.982299999999643
Yash_Mota --> 29.982299999999643
Yash_Narvekar92 --> 0.1668
Yaspal4428 --> 29.982299999999643
YasraRizvi --> 0.1668
yasserdent --> 0.20850000000000002
yatishturakhia --> 0.25020000000000003
YawaR_YS --> 0.1668
ybsarangi --> 0.

In [12]:
I[4]

-6497628.0